In [1]:
from SPARQLWrapper import SPARQLWrapper, RDFXML, JSON
import requests
import time

# Basic configuration
source = "https://wb.pcp-on-web.de/store-oker/research-ontology/"
result = "https://wb.pcp-on-web.de/store-oker/inferred-triples/"

# This function counts the genrated tripples of the research ontolgy (data:)
def count_triples(endpoint) :
    #count number of exiting triples
    sparql = SPARQLWrapper(endpoint+'sparql')
    sparql.setQuery('SELECT (COUNT(*) as ?Triples) WHERE { ?s ?p ?o}')
    sparql.setReturnFormat(JSON)
    json_results = sparql.query().convert()
    return json_results['results']['bindings'][0]['Triples']['value']
    

# This function executes a sparql construct query and imports the data to the research ontolgy (data:)
def query_sparql(query, endpoint_source, endpoint_result) :
    start_time = time.time()
    
    #perform construct Query
    sparql = SPARQLWrapper(endpoint_source+'sparql')
  
    #query += " LIMIT 10"
    sparql.setQuery(query)
    sparql.setReturnFormat(RDFXML)
    sparql.setOnlyConneg(True)
    resultsets = sparql.query().convert()
    result = resultsets.serialize(format='ntriples')
    
    text_lines = query.split("\n")
    info = text_lines[1]+"\n"+text_lines[2]
    
    # Add SPARQL INSERT to data to insert
    data = "INSERT DATA {\n" + result + "\n}"
   
    start_count = count_triples(endpoint_result)
    
    sparql = SPARQLWrapper(endpoint_result)
    sparql.setQuery(data)
    sparql.method = 'POST'
    sparql.query()

    end_count = count_triples(endpoint_result)

    output =  "\n"+info+"\n"+start_count + "-" + end_count + "\n--- " + str(time.time() - start_time) + " seconds ---"

    return output


In [2]:
data = """
# Inferred subClassOf relations 
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

CONSTRUCT   {

    ?resource rdf:type ?class1.
    ?resource rdf:type ?class2.
    ?resource rdf:type ?class3.
    ?resource rdf:type ?class4.
    ?resource rdf:type ?class5.
    ?resource rdf:type ?class6.
    ?resource rdf:type ?class7.
    ?resource rdf:type ?class8.
}
WHERE {
  
    ?resource rdf:type ?class.
    FILTER (contains(str(?resource),'data.pcp-on-web.de'))

    
    OPTIONAL {
        ?class rdfs:subClassOf ?class1.

        OPTIONAL {
            ?class1 rdfs:subClassOf ?class2.
            
            OPTIONAL {
                ?class2 rdfs:subClassOf ?class3.

                OPTIONAL {
                ?class3 rdfs:subClassOf ?class4.

                    OPTIONAL {
                    ?class4 rdfs:subClassOf ?class5.

                        OPTIONAL {
                        ?class5 rdfs:subClassOf ?class6.

                            OPTIONAL {
                            ?class6 rdfs:subClassOf ?class7.

                                OPTIONAL {
                                ?class7 rdfs:subClassOf ?class8.
    }}}}}}}}
    
}
"""
print(query_sparql(data,source,result))


# Inferred subClassOf relations 
#-----
0-10825
--- 3.5362918376922607 seconds ---
